In [1]:
import os
import math

In [2]:
os.getcwd()

'/home/wam/kood/emotions-detector/facial_emotion_recognition/notebook'

# get the data

## download the data

In [3]:
def show_folders_and_files(start_path='./', folder_limit=20, files_limit=5):
# ├ └  ─ │
    import os
    limit = 0
    last_root = None
    counter=0
    for root, dirs, files in os.walk(start_path):
        level = root.replace(start_path, '').count(os.sep) + 1
        if (limit:=limit+1) > folder_limit:
            print(f"{' ' * level+'.../'}")
            break
        if root == start_path:
            print(os.path.normpath(root)+'/')
        if root != start_path:
            print(f"{' ' * level}{os.path.basename(root)}/")
        if len(files) > files_limit:
            sample_number = 3 if files_limit > 3 else files_limit
            for f in files[:sample_number]:
                print(f"{' ' * (level+1)+'├ '}{f}")
            print(f"{' ' * (level+1)}├ ... {len(files)-sample_number-1} files")
        if len(files) < files_limit:
            for f in files[:-1]:
                print(f"{' ' * (level+1)+'├ '}{f}")
        if (fs:=files[-1:]):
            print(f"{' ' * (level+1)+'└ '}{fs[0]}")       
            
def show_folders(start_path, folder_limit=20):
    limit = 0
    for root, _, _ in os.walk(start_path):
        level = root.replace(start_path, '').count(os.sep) + 1
        if (limit:=limit+1) > folder_limit:
            print(f"{' ' * level+'.../'}")
            break
        if root == start_path:
            print(os.path.normpath(root)+'/')
        if root != start_path:
            print(f"{' ' * level}{os.path.basename(root)}/")

def show_files(start_path, files_limit=10):
    print(os.path.normpath(start_path)+'/')
    files = list(f for f in os.listdir(start_path) if os.path.isfile(os.path.join(start_path, f)))
    if len(files) > files_limit:
        sample_number = 3 if files_limit > 3 else files_limit
        for f in files[:sample_number]:
            print(f" ├ {f}")
        print(f" ├ ... {len(files)-sample_number} files")
    if len(files) < files_limit:
        for f in files[:-1]:
            print(f" ├ {f}")
    if (fs:=files[-1:]):
        print(f" └ {fs[0]}")       

In [4]:
data_url = 'https://assets.01-edu.org/ai-branch/project3/emotions-detector.zip'
data_folder_path = '../data/'

In [5]:
downloaded_data_file_name = 'downloaded_data.zip'

In [6]:
import requests
def fetch_file_stream(url, destination_folder_path, data_file_name, redownload=False):
    downloaded_file_path = destination_folder_path + data_file_name
    if not redownload:
        if os.path.exists(downloaded_file_path):
            return
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    if not os.path.exists(downloaded_file_path):
        response = requests.get(url, stream=True)
        with open(downloaded_file_path, 'wb') as downloading_file:
            for chunk in response.iter_content(chunk_size=128):
                downloading_file.write(chunk)
                
def unzip_files(zipped_file_path, destination_folder_path, check_path_already_exist=None):
    import zipfile
    import tarfile
    if check_path_already_exist:
        if os.path.exists(destination_folder_path):
            return
    try:
        with zipfile.ZipFile(zipped_file_path,"r") as zip_file:
            zip_file.extractall(destination_folder_path)
        return
    except:
        with tarfile.open(zipped_file_path, "r:gz") as gzip_file:
            gzip_file.extractall()
        return

In [7]:
fetch_file_stream(
    url=data_url,
    destination_folder_path=data_folder_path,
    data_file_name=downloaded_data_file_name,
    redownload=False,
)
show_folders_and_files(data_folder_path)

../data/
  └ downloaded_data.zip
 train/
  └ data.csv
 validate/
  └ data.csv
 test/
  └ data.csv
 unzipped_data/
  data/
   └ train.csv


## unzip the data

In [8]:
unzipped_data_folder_name = 'unzipped_data/'

In [9]:
unzip_files(
    zipped_file_path=data_folder_path+downloaded_data_file_name,
    destination_folder_path=data_folder_path+unzipped_data_folder_name,
    check_path_already_exist=f"{data_folder_path}{downloaded_data_file_name}"
)

In [10]:
show_files('../data/unzipped_data/')

../data/unzipped_data/


In [11]:
# datas_location = data_folder_path+unzipped_data_folder_name + 'data/'
datas_location = f"{data_folder_path}{unzipped_data_folder_name}data/"

data_file_names = list(
    file_name for file_name in os.listdir(datas_location)
    if os.path.isfile(os.path.join(datas_location, file_name)) and os.path.splitext(file_name)[1]=='.csv'
)

data_file_paths = list(
    datas_location + file_name 
    for file_name in data_file_names
)
data_file_keys = list( os.path.splitext(file_name)[0] for file_name in data_file_names )

# check the data

In [12]:
import pandas
import numpy

In [13]:
data_frames = {key: pandas.read_csv(path) for key,path in zip(data_file_keys, data_file_paths)}

In [14]:
for key,data_frame in data_frames.items():
    print(f"data_frame: {key}")
    print(f"  number of rows: {len(data_frame)}")
    print(f"  columns: {data_frame.columns}")
    print('')

data_frame: icml_face_data
  number of rows: 35887
  columns: Index(['emotion', ' Usage', ' pixels'], dtype='object')

data_frame: test
  number of rows: 7178
  columns: Index(['pixels'], dtype='object')

data_frame: test_with_emotions
  number of rows: 7178
  columns: Index(['Unnamed: 0', 'emotion', 'pixels'], dtype='object')

data_frame: train
  number of rows: 28709
  columns: Index(['emotion', 'pixels'], dtype='object')



## choose the data

### we pick up the icml_face_data
this seems to have all data set in one file 

In [15]:
all_data_frame = data_frames['icml_face_data']

In [16]:
def trim_column_names(data_frame):
    return data_frame.rename(
        columns={column: column.strip() for column in all_data_frame.columns}
    )
all_data_frame = trim_column_names(all_data_frame)

print(all_data_frame.info())
print('')
print(all_data_frame.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   Usage    35887 non-null  object
 2   pixels   35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None

   emotion     Usage                                             pixels
0        0  Training  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  Training  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  Training  231 212 156 164 174 138 161 173 182 200 106 38...


# assert the data

## hypothesis
according to the data description:
> The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

> train.csv contains two columns, "emotion" and "pixels". The "emotion" column contains a numeric code ranging from 0 to 6, inclusive, for the emotion that is present in the image. The "pixels" column contains a string surrounded in quotes for each image. The contents of this string a space-separated pixel values in row major order. test.csv contains only the "pixels" column and your task is to predict the emotion column.

> The training set consists of 28,709 examples. The public test set used for the leaderboard consists of 3,589 examples. The final test set, which was used to determine the winner of the competition, consists of another 3,589 examples.

> ...
- 28709 (train) + 3589 (validate) + 3589 (test) = 35887 total rows
- pixels represent a grayscale 48x48 image
  - assert each `pixels` values length is 48x48
  - assert each element of each `pixels` values fit uint8
- emotion is an unsigned integer between 0 and 6


### assert number of rows

In [17]:
all_data_frame['Usage'].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [18]:
assert len(all_data_frame) == 35887
assert len(all_data_frame[all_data_frame['Usage']=='Training']) == 28709
assert len(all_data_frame[all_data_frame['Usage']=='PublicTest']) == 3589

## for now we pursue the exploration with a sampling of the data

In [19]:
xs = [0,1,2,3]
ys = ['a','b','c']

In [20]:
[
    (x, y)
    for x in xs
    for y in ys
]

[(0, 'a'),
 (0, 'b'),
 (0, 'c'),
 (1, 'a'),
 (1, 'b'),
 (1, 'c'),
 (2, 'a'),
 (2, 'b'),
 (2, 'c'),
 (3, 'a'),
 (3, 'b'),
 (3, 'c')]

In [21]:

# def sampled_data_frame(data_frame, column_names, sampling):
#     df = data_frame
#     n = sampling
#     import pandas as pd
#     # Validate that the columns exist in the DataFrame
#     for col in column_names:
#         if col not in df.columns:
#             raise ValueError(f"Column '{col}' does not exist in the DataFrame.")
    
#     # Initialize an empty DataFrame to store the filtered rows
#     filtered_df = pd.DataFrame()

#     # Loop through each column name provided
#     for col in column_names:
#         # Get the unique values for the current column
#         unique_values = df[col].unique()

#         # For each unique value, sample up to 'n' rows and add to the filtered_df
#         for value in unique_values:
#             sampled_rows = df[df[col] == value].head(n)
#             filtered_df = pd.concat([filtered_df, sampled_rows])

#     # Drop duplicates that may have been added if columns overlap
#     filtered_df = filtered_df.drop_duplicates()

#     return filtered_df


In [22]:
all_data_frame.info

<bound method DataFrame.info of        emotion        Usage                                             pixels
0            0     Training  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1            0     Training  151 150 147 155 148 133 111 140 170 174 182 15...
2            2     Training  231 212 156 164 174 138 161 173 182 200 106 38...
3            4     Training  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4            6     Training  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...        ...          ...                                                ...
35882        6  PrivateTest  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883        3  PrivateTest  178 174 172 173 181 188 191 194 196 199 200 20...
35884        0  PrivateTest  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885        3  PrivateTest  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...
35886        2  PrivateTest  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...

[35887 rows x 3 col

In [65]:
for unique_Usage in all_data_frame['Usage'].unique():
    for unique_emotion in all_data_frame['emotion'].unique():
        print(f"{unique_Usage = } {unique_emotion = }")
        print(len(
            all_data_frame[(all_data_frame['Usage'] == unique_Usage) & (all_data_frame['emotion'] == unique_emotion)]
        ))
        #     all_data_frame[
        #         all_data_frame[all_data_frame['Usage']==unique_Usage]
        #     ][all_data_frame['emotion']==unique_emotion]
        # )

unique_Usage = 'Training' unique_emotion = 0
3995
unique_Usage = 'Training' unique_emotion = 2
4097
unique_Usage = 'Training' unique_emotion = 4
4830
unique_Usage = 'Training' unique_emotion = 6
4965
unique_Usage = 'Training' unique_emotion = 3
7215
unique_Usage = 'Training' unique_emotion = 5
3171
unique_Usage = 'Training' unique_emotion = 1
436
unique_Usage = 'PublicTest' unique_emotion = 0
467
unique_Usage = 'PublicTest' unique_emotion = 2
496
unique_Usage = 'PublicTest' unique_emotion = 4
653
unique_Usage = 'PublicTest' unique_emotion = 6
607
unique_Usage = 'PublicTest' unique_emotion = 3
895
unique_Usage = 'PublicTest' unique_emotion = 5
415
unique_Usage = 'PublicTest' unique_emotion = 1
56
unique_Usage = 'PrivateTest' unique_emotion = 0
491
unique_Usage = 'PrivateTest' unique_emotion = 2
528
unique_Usage = 'PrivateTest' unique_emotion = 4
594
unique_Usage = 'PrivateTest' unique_emotion = 6
626
unique_Usage = 'PrivateTest' unique_emotion = 3
879
unique_Usage = 'PrivateTest' unique

In [93]:
def sampled_data_frame(
    data_frame, 
    sampling_quantity, 
    to_be_preserved_column_names=['Usage','emotion'],
):
    unique_valuess = list(
        data_frame[column_name].unique()
        for column_name in column_names
    )
    import itertools
    products_of_values_to_preserve = itertools.product(*unique_valuess)
    result = pandas.DataFrame()   
    for one_set_of_unique_values in products_of_values_to_preserve:
        condition = pandas.Series([True] * len(data_frame))
        for column_name, unique_value in zip(column_names, one_set_of_unique_values):
            condition = condition & (data_frame[column_name]==unique_value)
        number_of_samples = sampling_quantity if sampling_quantity<condition.sum() else condition.sum()
        result = pandas.concat(
            [result, data_frame[condition].sample(n=number_of_samples)]
        )
    return result

In [94]:
sampled_all_data_frame = sampled_data_frame(
    data_frame=all_data_frame,
    sampling_quantity=60,
    column_names=['Usage','emotion']
)

# all_data_frame = pandas.concat([
#     all_data_frame[all_data_frame['Usage']=='Training'].sample(n=10),
#     all_data_frame[all_data_frame['Usage']=='PublicTest'].sample(n=10),
#     all_data_frame[all_data_frame['Usage']=='PrivateTest'].sample(n=10),
# ])

In [95]:
for unique_Usage in sampled_all_data_frame['Usage'].unique():
    for unique_emotion in sampled_all_data_frame['emotion'].unique():
        print(f"{unique_Usage = } {unique_emotion = }")
        print(len(
            sampled_all_data_frame[(sampled_all_data_frame['Usage'] == unique_Usage) & (sampled_all_data_frame['emotion'] == unique_emotion)]
        ))

unique_Usage = 'Training' unique_emotion = 0
60
unique_Usage = 'Training' unique_emotion = 2
60
unique_Usage = 'Training' unique_emotion = 4
60
unique_Usage = 'Training' unique_emotion = 6
60
unique_Usage = 'Training' unique_emotion = 3
60
unique_Usage = 'Training' unique_emotion = 5
60
unique_Usage = 'Training' unique_emotion = 1
60
unique_Usage = 'PublicTest' unique_emotion = 0
60
unique_Usage = 'PublicTest' unique_emotion = 2
60
unique_Usage = 'PublicTest' unique_emotion = 4
60
unique_Usage = 'PublicTest' unique_emotion = 6
60
unique_Usage = 'PublicTest' unique_emotion = 3
60
unique_Usage = 'PublicTest' unique_emotion = 5
60
unique_Usage = 'PublicTest' unique_emotion = 1
56
unique_Usage = 'PrivateTest' unique_emotion = 0
60
unique_Usage = 'PrivateTest' unique_emotion = 2
60
unique_Usage = 'PrivateTest' unique_emotion = 4
60
unique_Usage = 'PrivateTest' unique_emotion = 6
60
unique_Usage = 'PrivateTest' unique_emotion = 3
60
unique_Usage = 'PrivateTest' unique_emotion = 5
60
unique_U

In [96]:
sampled_all_data_frame[ (sampled_all_data_frame['emotion']==1) & (sampled_all_data_frame['Usage']=='PrivateTest')]

,emotion,Usage,pixels
33764,1,PrivateTest,130 129 130 131 136 139 138 144 146 144 144 15...
34519,1,PrivateTest,97 101 106 114 128 136 143 150 154 158 166 175...
34022,1,PrivateTest,66 71 77 81 83 87 89 100 112 126 139 144 140 1...
34593,1,PrivateTest,0 8 29 49 63 72 70 64 69 74 93 132 159 141 166...
32923,1,PrivateTest,252 255 253 254 254 254 252 255 228 191 189 19...
35391,1,PrivateTest,195 178 195 201 190 138 124 93 60 72 86 96 103...
32525,1,PrivateTest,74 41 42 61 51 46 51 34 31 30 28 34 53 52 71 7...
34958,1,PrivateTest,255 224 172 197 197 200 201 204 209 211 209 21...
34780,1,PrivateTest,37 43 23 34 72 105 84 55 48 68 120 108 90 86 1...
35312,1,PrivateTest,70 88 94 86 86 92 107 114 116 119 123 131 131 ...


In [63]:
all_data_frame[ (all_data_frame['emotion']==1) & (all_data_frame['Usage']=='PrivateTest')]

,emotion,Usage,pixels
32332,1,PrivateTest,22 24 20 13 27 31 51 114 143 161 168 172 174 1...
32476,1,PrivateTest,75 66 63 68 78 86 117 148 161 169 170 175 179 ...
32487,1,PrivateTest,20 15 13 16 18 56 91 111 149 153 167 178 187 1...
32525,1,PrivateTest,74 41 42 61 51 46 51 34 31 30 28 34 53 52 71 7...
32531,1,PrivateTest,226 139 61 62 47 38 24 19 16 25 50 81 100 115 ...
32546,1,PrivateTest,57 58 57 61 49 19 12 39 66 82 157 217 209 213 ...
32681,1,PrivateTest,215 213 208 207 199 197 177 106 81 73 70 72 72...
32717,1,PrivateTest,49 43 44 47 51 49 46 43 45 47 51 43 37 49 49 5...
32912,1,PrivateTest,16 20 26 19 18 14 57 75 135 64 64 46 20 78 56 ...
32923,1,PrivateTest,252 255 253 254 254 254 252 255 228 191 189 19...


In [ ]:
1/0

In [ ]:
del data_frames

## assert labels

In [ ]:
def assert_emotion_series_represent_anumber_from_0_to_6(emotion_number_label_series):
    [
        number if 0<=number<=6 else 1/0
        for number in emotion_number_label_series
    ]

In [ ]:
assert_emotion_series_represent_anumber_from_0_to_6(all_data_frame['emotion'])

## assert image format

In [ ]:
def assert_string_series_represent_48x48_uint8_series(string_series):
    [
        [
            int(numerale) if 0<=int(numerale)<=255 else 1/0
            for numerale in numerales.split() 
        ] if len(numerales.split())==48*48 else 1/0
        for numerales in string_series
    ]

In [ ]:
assert_string_series_represent_48x48_uint8_series(all_data_frame['pixels'])

# format data

## format set

In [ ]:
def reformatted_set_series(set_series):
    old_set_names = ['Training', 'PublicTest', 'PrivateTest']
    new_set_names = ['train', 'validate', 'test']
    return (
        set_series.astype('category').cat.rename_categories(
                {old_name: new_name for old_name,new_name in zip(old_set_names, new_set_names)}
    ) )

In [ ]:
all_data_frame['set'] = reformatted_set_series(all_data_frame['Usage'])
all_data_frame = all_data_frame.drop(columns='Usage')

In [ ]:
all_data_frame.info()

## format labels

In [ ]:
# all_data_frame['set']#.astype(numpy.uint8)

In [ ]:
all_data_frame['emotion'].head()

In [ ]:
def reformatted_emotion_series(emotion_series):
    return emotion_series.astype(numpy.uint8).astype('category')

In [ ]:
all_data_frame['emotion'] = reformatted_emotion_series(all_data_frame['emotion'])

In [ ]:
all_data_frame.info()

## format images

In [ ]:
def integers_series_from_numerales_string_series(numbers_string_series, number_type=numpy.int64):
    return pandas.Series(
        data=(
            numpy.array( [number_type(numerale) for numerale in numerales.split()] ) 
            for numerales in numbers_string_series
        ),
        index=numbers_string_series.index,
    )

In [ ]:
def bytes_series_from_uint8s_series(uint8s_series):
    return pandas.Series(
        data=(uint8s.tobytes() for uint8s in uint8s_series)
        ,
        index=uint8s_series.index
    )

In [ ]:
def matrix_series_from_single_array_series(integers_array_series, width=None, height=None):
    return pandas.Series(
        data=[
            numpy.array(
                [pixel for pixel in pixels]
            ).reshape(width,height)
            for pixels in integers_array_series
        ],
        index=integers_array_series.index
    )

In [ ]:
def uint8_matrix_series_from_numerales_string_series(numerales_string_series):
    uint8s_series = integers_series_from_numerales_string_series(
        all_data_frame['pixels'],
        number_type=numpy.uint8,
    )
    uint8s_matrix_series = matrix_series_from_single_array_series(
        uint8s_series,
        width=48, height=48,
    )
    return uint8s_matrix_series

In [ ]:
all_data_frame['pixels_as_matrix'] = uint8_matrix_series_from_numerales_string_series(all_data_frame['pixels'])

In [ ]:
def bytes_series_from_numerales_string_series(numerales_string_series):
    uint8s_series = integers_series_from_numerales_string_series(
        all_data_frame['pixels'],
        number_type=numpy.uint8,
    )
    bytes_series = bytes_series_from_uint8s_series(
        uint8s_series,
    )
    return bytes_series

In [ ]:
all_data_frame['pixels_as_bytes'] = bytes_series_from_numerales_string_series(all_data_frame['pixels'])

In [ ]:
all_data_frame = all_data_frame.drop(columns=['pixels'])

### as a path to png saved image
middle step as uint8 2d matrix

In [ ]:
all_data_frame.info()

*split data into train, validate and test set*

*set up folder structure*
- train/
  - data.csv
  - images/
    - {id}.png
    - ...
- validate/
  - data.csv
  - images/
    - {id}.png
    - ...
- test/
  - data.csv
  - images/
    - {id}.png
    - ...

In [ ]:
from PIL import Image

def save_image_to_png(image_matrix, destination_folder_path, image_file_name):
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    image_png = Image.fromarray(image_matrix, 'L')
    image_png.save(image_file_path:=f"{destination_folder_path}{image_file_name}")
    return image_file_path

def saved_image_file_path_series_from_data_frame(
    set_series,
    emotion_series,
    image_matrix_series,
    data_folder_path='./',
    emotion_names=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'],
):
    # determine the file path where we gonna save each image from its set and emotion label(name)
    image_folder_paths_and_image_file_names = (
        (
            f"{data_folder_path}{set_name}/images/{emotion_names[emotion_number]}/"
            ,
            f"{index}.png"
        ) 
        for set_name, emotion_number, index 
        in zip(set_series, emotion_series, image_matrix_series.index) 
    )
    # save images as png
    image_file_paths = list( 
        save_image_to_png(image_matrix, destination_folder_path, image_file_name)
        for image_matrix, (destination_folder_path, image_file_name)
        in zip(image_matrix_series, image_folder_paths_and_image_file_names)
    )
    # return saved file paths as series
    saved_paths = pandas.Series(
        data=image_file_paths,
        index=image_matrix_series.index,
    )
    return saved_paths


In [ ]:
all_data_frame['image_file_path'] = saved_image_file_path_series_from_data_frame(
    set_series=all_data_frame['set'],
    emotion_series=all_data_frame['emotion'],
    image_matrix_series=all_data_frame['pixels_as_matrix'],
    data_folder_path=data_folder_path,
)
all_data_frame = all_data_frame.drop(columns='pixels_as_matrix')

In [ ]:
# def saved_image_file_path_series_from_data_frame(
#     data_frame,
#     set_column_name='set',
#     emotion_column_name='emotion',
#     image_matrix_column_name='pixels_as_matrix',
#     data_folder_path='./',
#     emotion_names=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'],
# ):
#     import utils
#     saving_folder_path_and_file_name_series=data_frame.apply(
#         lambda row: (
#             f"{data_folder_path}"
#             f"{row[set_column_name]}/images/{emotion_names[row[emotion_column_name]]}/"
#             ,
#             f"{row.name}.png"
#         ),
#         axis=1,
#     )
#     list(
#         utils.save_image_to_png(image_matrix, destination_folder_path, image_file_name)
#         for image_matrix, (destination_folder_path, image_file_name)
#         in zip(data_frame[image_matrix_column_name], saving_folder_path_and_file_name_series)
#     )
#     saved_paths = pandas.Series(
#         data=(
#             f"{folder_path}{file_name}"
#             for (folder_path,file_name) 
#             in saving_folder_path_and_file_name_series
#         ),
#         index=data_frame.index
#     )
#     return saved_paths

In [ ]:
# all_data_frame['image_file_path'] = saved_image_file_path_series_from_data_frame(
#     data_frame=all_data_frame,
#     set_column_name='set',
#     emotion_column_name='emotion',
#     image_matrix_column_name='pixels_as_matrix',
#     data_folder_path=data_folder_path,
# )
# all_data_frame = all_data_frame.drop(columns='pixels_as_matrix')

In [ ]:
all_data_frame.head(1)

In [ ]:
show_folders_and_files(data_folder_path)

In [ ]:
def saved_data_set_path_by_name_paths_from_dataframe(data_frame, data_folder_path
                                                    ):
    def saved_data_set_frame_path(data_set_frame, saving_file_path):
        data_set_frame.to_csv(
            saving_file_path,
            columns=['emotion','image_file_path','pixels_as_bytes'],
            index=False,
        )
        return saving_file_path
        
    return {
        set_name:saved_data_set_frame_path(
            data_frame[data_frame['set'] == set_name],
            f"{data_folder_path}{set_name}/data.csv",
        ) 
        for set_name in data_frame['set'].unique()
    }

In [ ]:
saved_data_frame_paths = saved_data_set_path_by_name_paths_from_dataframe(all_data_frame, data_folder_path)

In [ ]:
saved_data_frame_paths

## check that we correctly saved the picutres

In [ ]:
def show_image_from_path(image_file_path):
    from IPython.display import display
    from PIL import Image
    image = Image.open(image_file_path)
    display(image)

In [ ]:
def display_sample_of_each_set_of_each_emotion(
    data_frame,
    data_folder_path, 
    emotion_names=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
):
    for set_name in data_frame['set'].unique():
        for emotion_index in data_frame['emotion'].unique():
            emotion_name = emotion_names[emotion_index]
            data_of_the_set = data_frame[ data_frame['set']==set_name ] 
            data_of_the_set_and_the_emotion = data_of_the_set[ data_of_the_set['emotion']==emotion_index ]
            print(f"{data_folder_path}{set_name}/images/{emotion_name}/")
            n_rows = len(data_of_the_set_and_the_emotion)
            if n_rows > 0:
                n_samples = n_rows if n_rows < 3 else 3
                indices = data_of_the_set_and_the_emotion.sample(n=n_samples).index
                for index in indices:
                    image_path = f"{data_folder_path}{set_name}/images/{emotion_name}/{index}.png"
                    show_image_from_path(image_path)

In [ ]:
display_sample_of_each_set_of_each_emotion(all_data_frame, data_folder_path)

In [ ]:
# import shutil
# for data_set_folder_path in data_set_folder_paths:
#     shutil.rmtree(data_set_folder_path)

In [ ]:
# import shutil
# shutil.rmtree(data_folder_path)